### Parses all champ stats from a hosted server
This allows us to retrieve descriptions for all abilities and retrieve ability related info.

In [3]:
import os

CHAMP_STATS_URL = "https://cdn.merakianalytics.com/riot/lol/resources/latest/en-US/champions/"

In [3]:
import requests
from bs4 import BeautifulSoup

page = requests.get(CHAMP_STATS_URL)
soup = BeautifulSoup(page.content, "html.parser")
champ_hrefs = soup.find_all(href=lambda x: "json" in x if x else False)
champ_jsons = [champ_href.contents[0] for champ_href in champ_hrefs]
champ_json_urls = [f"{CHAMP_STATS_URL}{champ_json}" for champ_json in champ_jsons]

In [4]:
import urllib.request, json

character_abilities = {}
for champ_json_url in champ_json_urls:
    with urllib.request.urlopen(champ_json_url) as url:
        data = json.load(url)
        name = data['name']
        abilities = data['abilities']
        character_abilities[name] = abilities

In [5]:
with open("characters.json", "w+") as f:
    json.dump(character_abilities, f)

### Video scraper scripts
Want to download all the videos that are in the LoL wiki.

In [5]:
from bs4 import BeautifulSoup
import requests

LOL_WIKI = "https://leagueoflegends.fandom.com"
LIST_OF_CHAMPS = LOL_WIKI + "/wiki/List_of_champions"

champs_soup = BeautifulSoup(requests.get(url=LIST_OF_CHAMPS).content, 'html.parser')

In [6]:
nested_links = champs_soup.select('span > span > a')
champ_links = list(set([link['href'] for link in nested_links]))
champ_links = [f"{LOL_WIKI}{link}" for link in champ_links]
champ_links

['https://leagueoflegends.fandom.com/wiki/Blitzcrank/LoL',
 'https://leagueoflegends.fandom.com/wiki/Kayn/LoL',
 'https://leagueoflegends.fandom.com/wiki/Urf',
 'https://leagueoflegends.fandom.com/wiki/Rell/LoL',
 'https://leagueoflegends.fandom.com/wiki/Leona/LoL',
 'https://leagueoflegends.fandom.com/wiki/Yuumi/LoL',
 'https://leagueoflegends.fandom.com/wiki/Zoe/LoL',
 'https://leagueoflegends.fandom.com/wiki/Wukong/LoL',
 'https://leagueoflegends.fandom.com/wiki/Tabu',
 'https://leagueoflegends.fandom.com/wiki/Tristana/LoL',
 'https://leagueoflegends.fandom.com/wiki/Fiora/LoL',
 'https://leagueoflegends.fandom.com/wiki/Garen/LoL',
 'https://leagueoflegends.fandom.com/wiki/Shen/LoL',
 'https://leagueoflegends.fandom.com/wiki/Nidalee/LoL',
 'https://leagueoflegends.fandom.com/wiki/Lissandra/LoL',
 'https://leagueoflegends.fandom.com/wiki/Kog%27Maw/LoL',
 'https://leagueoflegends.fandom.com/wiki/Cho%27Gath/LoL',
 'https://leagueoflegends.fandom.com/wiki/Rakan/LoL',
 'https://leagueofle

In [12]:
import ffmpeg

def ogv_to_webm(input_file, output_file):
    try:
        ffmpeg.input(input_file).output(output_file, format='webm').run()
        print(f"Converted {input_file} to {output_file}")
    except ffmpeg.Error as e:
        print(f"Error: {e.stderr.decode()}")


In [14]:
import glob
import re

SAVE_DIR = "Data/Videos"
os.makedirs(SAVE_DIR, exist_ok=True)

for url in champ_links:
    champ_name = url.split("/")[-2]
    if glob.glob(f"{SAVE_DIR}/{champ_name}*.webm"):
        print(f"Already downloaded videos for {champ_name}")
        continue
    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    video_links = soup.select('video')
    downloads = [link['src'] for link in video_links]
    if len(downloads) == 0:
        print(f"No links found for: {champ_name}")
        continue
    
    filename_pattern = re.compile(r'/([^/]+\.(webm|ogv))')
    for download in downloads:
        match = filename_pattern.search(download)
        if not match:
            print(f"Failed to extract filename for URL: {download}")
            continue
       
        file_name = match.group(1).replace('Video', '')
        file_path = os.path.join(SAVE_DIR, file_name)
        file_type = match.group(2).split('.')[-1]
        print(f"Downloading: {download}")
        download_req = requests.get(download, stream=True)
        if download_req.status_code != 200:
            print(f"Failed to download: {download} (Status code: {download_req.status_code})")
            continue
        with open(file_path, 'wb') as file:
            for chunk in download_req.iter_content(chunk_size=1024):
                if chunk:  # Filter out keep-alive chunks
                    file.write(chunk)
        print(f"Saved to {file_path}")              
        if file_type == "ogv":
            ogv_to_webm(file_path, file_path.replace(".ogv", ".webm"))

Downloading: https://static.wikia.nocookie.net/leagueoflegends/images/d/d8/Blitzcrank_IVideo.ogv/revision/latest?cb=20160531120918
Saved to Data/Videos/Blitzcrank_I.ogv


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with clang version 18.1.8
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1732155217949/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1732155217949/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --disable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --e

Converted Data/Videos/Blitzcrank_I.ogv to Data/Videos/Blitzcrank_I.webm
Downloading: https://static.wikia.nocookie.net/leagueoflegends/images/e/e6/Blitzcrank_QVideo.ogv/revision/latest?cb=20160531120921
Saved to Data/Videos/Blitzcrank_Q.ogv


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with clang version 18.1.8
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1732155217949/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1732155217949/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --disable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --e

Converted Data/Videos/Blitzcrank_Q.ogv to Data/Videos/Blitzcrank_Q.webm
Downloading: https://static.wikia.nocookie.net/leagueoflegends/images/1/1e/Blitzcrank_WVideo.ogv/revision/latest?cb=20160531120935
Saved to Data/Videos/Blitzcrank_W.ogv


...

Converted Data/Videos/Braum_W.ogv to Data/Videos/Braum_W.webm
Downloading: https://static.wikia.nocookie.net/leagueoflegends/images/d/de/Braum_EVideo.ogv/revision/latest?cb=20181108200228
Saved to Data/Videos/Braum_E.ogv


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with clang version 18.1.8
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1732155217949/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1732155217949/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --disable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --e

Converted Data/Videos/Braum_E.ogv to Data/Videos/Braum_E.webm
Downloading: https://static.wikia.nocookie.net/leagueoflegends/images/d/d9/Braum_RVideo.ogv/revision/latest?cb=20181108200336
Saved to Data/Videos/Braum_R.ogv


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with clang version 18.1.8
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1732155217949/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1732155217949/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --disable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --e

Converted Data/Videos/Braum_R.ogv to Data/Videos/Braum_R.webm
No links found for: Nasus
No links found for: Jax
No links found for: Aurelion_Sol
No links found for: Jinx
No links found for: wiki
No links found for: Udyr
No links found for: K%27Sante
No links found for: Sivir
No links found for: Olaf
